In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

%matplotlib inline

df = pd.read_csv('dataset_preprocessing.csv', index_col=0)
df.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_N,Embarked_Q,Embarked_S,Survived
0,0.271174,0.125,0.0,0.014151,False,False,True,False,True,False,False,False,True,0
1,0.472229,0.125,0.0,0.139136,True,False,False,True,False,True,False,False,False,1
2,0.321438,0.000,0.0,0.015469,False,False,True,True,False,False,False,False,True,1
3,0.434531,0.125,0.0,0.103644,True,False,False,True,False,False,False,False,True,1
4,0.434531,0.000,0.0,0.015713,False,False,True,False,True,False,False,False,True,0


In [2]:
label = df["Survived"]
df = df.drop(["Survived"], axis=1)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.25, random_state=1234)

### 0. 일반적인 모델 학습

In [4]:
# 모델 초기화
n_estimator = 300
random_state = 1234

model = RandomForestClassifier(n_estimators=n_estimator, random_state=random_state)
# 모델 학습
model.fit(X_train, y_train)
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
prf = precision_recall_fscore_support(y_test, y_pred, average='binary')

In [5]:
# 정확도 출력
accuracy

0.8116591928251121

In [6]:
prf

(0.788235294117647, 0.7362637362637363, 0.7613636363636364, None)

### 1. MLflow를 활용한 모델 학습 Tracking
* custom logging (직접 로깅) 을 통한 모델 학습 기록 관리
* auto logging을 활용한 모델 학습 기록 관리
* auto loggine + custom logging

##### 1) Custom logging(직접 로깅)을 통한 모델 학습 기록 관리

In [7]:
import mlflow

'''
	To do
    tracking uri를 셋팅하기 -> local host로 진행 
    experiment 생성하기 -> hellomlflow!
    experiment 셋팅하기 -> hellomlflow!
'''

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.create_experiment("hellomlflow!")


'745210100665217220'

In [8]:
mlflow.set_experiment("hellomlflow!")

<Experiment: artifact_location='mlflow-artifacts:/745210100665217220', creation_time=1742631717739, experiment_id='745210100665217220', last_update_time=1742631717739, lifecycle_stage='active', name='hellomlflow!', tags={}>

In [9]:
'''
	To do
	mlflow 활용해서 Custom Logging을 진행 
    아래 함수를 이용해 logging을 진행
		1) mlflow.log_param()
			n_estimator
		2) mlflow.log_model()
			save model
    3) mlflow.log_metric()
			metric -> accuracy, precision, recall, f1score

'''

n_estimator = 80
random_state = 1234

with mlflow.start_run():
    model = RandomForestClassifier(n_estimators=n_estimator, random_state=random_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    prf = precision_recall_fscore_support(y_test, y_pred, average="binary")
    mlflow.log_param("n_estimator", n_estimator)
    mlflow.log_metric("accuracy_on_test", accuracy)
    mlflow.log_metric("precision_on_test", prf[0])
    mlflow.log_metric("recall_on_test", prf[1])
    mlflow.log_metric("f1score_on_test", prf[2])
    mlflow.sklearn.log_model(model, "model")

2025/03/22 17:22:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run clumsy-shrew-937 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/6ccd366963e4443d84d7a33e7f387b17
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220


##### 2) auto logging을 활용한 모델 학습 기록 관리

In [10]:
# Mlflow Sklearn을 활용해서 모델 및 메트릭 자동 기록! 
'''
	To do 
	use autolog() on mlflow 
'''

mlflow.sklearn.autolog()

In [11]:
# 모델 초기화
n_estimator = 77
random_state = 2222

model = RandomForestClassifier(n_estimators=n_estimator, random_state=random_state)
# 모델 학습
model.fit(X_train, y_train)
# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)
# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
prf = precision_recall_fscore_support(y_test, y_pred, average='binary')

2025/03/22 17:22:34 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'edf5d1a9a7484f0396e5042473cd84c4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run caring-horse-800 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/edf5d1a9a7484f0396e5042473cd84c4
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220


### 3) auto logging + custom logging을 활용한 모델 학습 관리

In [12]:
'''
	To do
	auto logging에서 수집되지 않는 정보 추가로 logging 하기 
		precision, recall, f1score, accuracy for test data set 

'''

with mlflow.start_run():
    n_estimator = 400 
    random_state = 7777
    max_depth = 2
    model = RandomForestClassifier(n_estimators=n_estimator, random_state=random_state, max_depth=max_depth)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    prf = precision_recall_fscore_support(y_test, y_pred, average="binary")
    mlflow.log_metric("precision_on_test", prf[0])
    mlflow.log_metric("recall_on_test", prf[1])
    mlflow.log_metric("f1score_on_test", prf[2])
    mlflow.log_metric("accuracy_on_test", accuracy)



🏃 View run worried-bee-726 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/1591682db1ef4b03905315ec9d6bc82c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220


### 2. MLflow를 활용한 모델 최적화 Tracking
* 직접 parameter 변경해가며 모델 최적화
* GridSearch 혹은 RandomSearch를 활용하여 모델 최적화


##### 1) 직접 parameter 변경해가며 모델 최적화

In [13]:
def train_model_with_hyperparameters(n_estimator, max_depth, max_feature):
    with mlflow.start_run():
        model = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, max_features=max_feature)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        prf = precision_recall_fscore_support(y_test, y_pred, average='binary')
        mlflow.log_metric("precision_on_test", prf[0])
        mlflow.log_metric("recall_on_test", prf[1])
        mlflow.log_metric("f1score_on_test", prf[2])
        mlflow.log_metric("accuracy_on_test", accuracy)

In [14]:
n_estimators = [50, 100, 150, 200]
max_depths = [2, 5, 10]
max_features = [5, 8, 10, 13]

'''
    To do
    run model with hyper-parameter combination 
'''

for n_estimator in n_estimators:
    for max_depth in max_depths:
        for max_feature in max_features:
            train_model_with_hyperparameters(n_estimator, max_depth, max_feature)

🏃 View run bold-ape-851 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/172a95ea94064613b0a062415baef27e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run funny-mole-384 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/a72f921e89974f2896d8cdee09bf9f19
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run blushing-fawn-236 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/68bd2a2ca27a4e7594350bb712c11e49
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run agreeable-turtle-849 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/824b0d9aaa5f447f9f8dd1154d57e040
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run nebulous-robin-632 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/1437f4dab824416ca2e096b75e310100
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/74521010

##### 2) GridSearch 혹은 RandomSearch를 활용하여 모델 최적화


In [15]:
from sklearn.model_selection import GridSearchCV


'''
	To do
	run GridSearchCV with mlflow 
'''

with mlflow.start_run():
    model_grid = GridSearchCV(RandomForestClassifier(), {"n_estimators":[100,200], "max_depth":[20,25]})
    model_grid.fit(X_train, y_train)
    y_pred = model_grid.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    prf = precision_recall_fscore_support(y_test, y_pred, average="binary")
    mlflow.log_metric("precision_on_test", prf[0])
    mlflow.log_metric("recall_on_test", prf[1])
    mlflow.log_metric("f1score_on_test", prf[2])
    mlflow.log_metric("accuracy_on_test", accuracy)


2025/03/22 17:38:41 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.


🏃 View run legendary-jay-846 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/a0113f2054924b1cbe40d2002949a8a3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run magnificent-hen-179 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/486d59ec5421434cbf23ffe111f7f748
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run dazzling-frog-245 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/0ba0ed47cd264ae991e1ba3005a2ab01
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run delightful-moth-120 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/ad7b277edcb0497382abb2c18f0b67c7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/745210100665217220
🏃 View run loud-tern-252 at: http://127.0.0.1:5000/#/experiments/745210100665217220/runs/bb002988d23a4e1da802870f713e060e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7452